In [146]:
#Беру только по 2500 предложений, потому что, если брать больше, происходит Memory Error. Видимо, мой комп совсем плох.

In [69]:
import math
from math import log

In [70]:
import csv
with open('quora_question_pairs_rus.csv', 'r', encoding = 'utf-8') as r:
    reader = csv.reader(r)
    file = list(reader)

In [71]:
import pymorphy2 as pm2
pmm = pm2.MorphAnalyzer()
import re

In [72]:
russian_stopwords = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как',
                         'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у',
                         'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот',
                         'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда',
                         'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть',
                         'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь',
                         'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где',
                         'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам',
                         'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет',
                         'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой',
                         'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем',
                         'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда',
                         'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над',
                         'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая',
                         'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою',
                         'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой',
                         'им', 'более', 'всегда', 'конечно', 'всю', 'между']

In [185]:
docs = []
for indx, row in enumerate(file):
    if indx > 0:
        docs.append([row[2], row[3]])

In [6]:
#corpus = []
#for indx, row in enumerate(file):
#    if indx > 0:
#        doc = row[2]
#        doc = re.sub(r'[^\w\s]', '', doc)
#        doc = re.sub(r'\d', '', doc)
#        doc = re.sub(r'[A-Za-z]', '', doc)
#        doc_arr = [pmm.normal_forms(x)[0] for x in doc.split() if x not in russian_stopwords]
#        doc = ' '.join(doc_arr)
#        corpus.append(doc)

In [8]:
#queries = []
#for indx, row in enumerate(file):
#    if indx > 0:
#        query = row[1]
#        query = re.sub(r'[^\w\s]', '', query)
#        query = re.sub(r'\d', '', query)
#        query = re.sub(r'[A-Za-z]', '', query)
#        query_arr = [pmm.normal_forms(x)[0] for x in query.split() if x not in russian_stopwords]
#        query = ' '.join(query_arr)
#        queries.append(query)

In [163]:
#with open('corpus.txt', 'r', encoding = 'utf-8') as r:
#    corpus = r.readlines()

In [178]:
#with open('queries.txt', 'r', encoding = 'utf-8') as r:
#    queries = r.readlines()

In [164]:
#corp = corpus[:2500]

In [179]:
#quer = queries[:2500]

In [169]:
#with open('corp.txt', 'w', encoding = 'utf-8') as w:
#    for i in corp:
#        w.write(i)

In [180]:
#with open('quer.txt', 'w', encoding = 'utf-8') as w:
#    for i in quer:
#        w.write(i)

In [171]:
with open('corp.txt', 'r', encoding = 'utf-8') as r:
    corp = r.readlines()

In [183]:
with open('quer.txt', 'r', encoding = 'utf-8') as r:
    quer = r.readlines()

## Подготовительная работа. Подсчет переменных для формулы

In [78]:
#кол-во документов в коллекции
N = len(corp)
N

2500

In [79]:
#длины документов
lens_d = []
for doc in corp:
    doc = doc.split()
    lens_d.append(len(doc))
lens_d[:5]

[7, 5, 5, 4, 8]

In [80]:
#средняя длина документа
avgdl = sum(lens_d)/len(lens_d)
avgdl

5.9084

In [81]:
#константы
k = 2
b = 0.75

In [82]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
import numpy as np

In [83]:
X = vectorizer.fit_transform(corp)
#матрица с количеством вхождений слов
matrix = X.toarray()

In [84]:
#обратная матрица
rev_matrix = np.transpose(matrix)

In [186]:
words = vectorizer.get_feature_names()

In [86]:
#кол-во слов
len(rev_matrix)

4304

In [87]:
#кол-во документов, где есть слово
number_of_docs = []
for word in rev_matrix:
    count = 0
    for i in word:
        if i != 0:
            count += 1
    number_of_docs.append(count)

In [88]:
TFs = []
for word in words:
    arr = []
    for sent in corp:
        TF = 0
        sent = sent.split()
        if word in sent:
            num = 0
            for s in sent:
                if word == s:
                    num += 1
            TF = num / len(sent)
        arr.append(TF)
    TFs.append(arr)

In [89]:
len(TFs)

4304

# Задача 1

## Способ №1. Подсчет метрики по формуле для каждой пары слово-документ

In [90]:
quer[6]

'мочь хороший геолог\n'

In [96]:
#проверяю время работы только на 1-м запросе, потому что на 2500 будет работать почти 5 часов...

In [95]:
%%time
q = quer[6]
result = []
for index, sent in enumerate(corp):
    bms = []
    for i in q.split():
        for indx, word in enumerate(words):
            if i == word:
                n = number_of_docs[indx]
                TF_arr = TFs[indx]
        l = len(sent.split())
        TF = TF_arr[index]
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        bms.append(bm)
    res = sum(bms)
    result.append(res)

Wall time: 7.34 s


## Способ №2. Подсчет метрики через умножение матрицы на вектор

In [97]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [105]:
import numpy as np

In [106]:
%%time
q = quer[6]
bms = []
for word in q.split():
    for indx, w in enumerate(words):
        if word == w:
            bm = bm_mtrx[indx]
            bms.append(bm)
vector = sum(np.array(bms))

Wall time: 4.99 ms


In [107]:
#Итог: способ с матрицей быстрее в 1829 раз.

# Задача 2

In [145]:
#ДИСКЛЕЙМЕР!!! т.к. мой корпус состоит всего из 2500 предложений, в нем нет слов из запроса "рождественские каникулы". Поэтому я взяла другое словосочетание.

In [139]:
q_input = 'день рождения'
q_arr = [pmm.normal_forms(x)[0] for x in q_input.split() if x not in russian_stopwords]
bms_q = []
for word in q_arr:
    for indx, w in enumerate(words):
        if word == w:
            bm = bm_mtrx[indx]
            bms_q.append(bm)

In [140]:
v = sum(np.array(bms_q))
v

array([0., 0., 0., ..., 0., 0., 0.])

In [141]:
for_range = list(zip(v, docs))

In [142]:
sort_range = sorted(for_range, reverse=True)

In [143]:
for indx, row in enumerate(sort_range):
    if indx < 10:
        print(row[1][0], ': ', row[0])

если бы у вас была возможность быть кем-то еще на один день, кто бы вы были :  2.7763629925786444
будет есть 2 000 калорий в день, чтобы я похудел :  2.7763629925786444
можно сжечь 1000 калорий каждый день, если да, как :  1.8811307833920714
что вы можете рассказать мне относительно этой диаграммы рождения? :  1.8637841806164563
насколько важно быть первым, кто пожелает кому-то счастливого дня рождения :  1.4889391878490699
что произойдет, если диктатура сохранится и в наши дни :  1.349112540919825
у меня есть мания, если я слушаю музыку в течение всего дня :  1.349112540919825
сколько раз нужно заниматься сексом в день :  1.349112540919825
сколько раз в день перекрывают руки часы :  1.349112540919825
сколько минут йоги можно сделать за один день :  1.349112540919825


In [150]:
if_success = []
for indx, row in enumerate(sort_range):
    if indx < 5:
        if row[1][1] == '1':
            if_success.append('wow')
if 'wow' in if_success:
    print('Алгоритм точен!')
else:
    print('Алгоритм не точен.')

Алгоритм точен!


# Задача 3

## BM25, b=0.75

In [148]:
b = 0.75

In [149]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [151]:
q_input = 'день рождения'
q_arr = [pmm.normal_forms(x)[0] for x in q_input.split() if x not in russian_stopwords]
bms_q = []
for word in q_arr:
    for indx, w in enumerate(words):
        if word == w:
            bm = bm_mtrx[indx]
            bms_q.append(bm)
v = sum(np.array(bms_q))
for_range = list(zip(v, docs))
sort_range = sorted(for_range, reverse=True)
if_success = []
for indx, row in enumerate(sort_range):
    if indx < 5:
        if row[1][1] == '1':
            if_success.append('wow')
if 'wow' in if_success:
    print('Алгоритм точен!')
else:
    print('Алгоритм не точен.')

Алгоритм точен!


## BM15, b=0

In [152]:
b = 0

In [153]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [154]:
q_input = 'день рождения'
q_arr = [pmm.normal_forms(x)[0] for x in q_input.split() if x not in russian_stopwords]
bms_q = []
for word in q_arr:
    for indx, w in enumerate(words):
        if word == w:
            bm = bm_mtrx[indx]
            bms_q.append(bm)
v = sum(np.array(bms_q))
for_range = list(zip(v, docs))
sort_range = sorted(for_range, reverse=True)
if_success = []
for indx, row in enumerate(sort_range):
    if indx < 5:
        if row[1][1] == '1':
            if_success.append('wow')
if 'wow' in if_success:
    print('Алгоритм точен!')
else:
    print('Алгоритм не точен.')

Алгоритм точен!


## BM11, b=1

In [161]:
b = 1

In [162]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

ZeroDivisionError: float division by zero

In [160]:
#BM11 не сработало из-за запрета деления на 0.